In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from dataclasses import dataclass

@dataclass 
class EmailContext:
  email_address: str = "eliel@example.com"
  password: str = "password123"

In [3]:
from langchain.agents import AgentState

class AuthenticatedState(AgentState):
    authenticated: bool

In [4]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def check_inbox() -> str:
    """Check the inbox for recent emails"""
    return """
    Hi Julie, 
    I'm going to be in town next week and was wondering if we could grab a coffee?
    - best, Jane (jane@example.com)
    """

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an response email"""
    return f"Email sent to {to} with subject {subject} and body {body}"

@tool
def authenticate(email: str, password: str, runtime: ToolRuntime) -> Command:
    """Authenticate the user with the given email and password"""
    if email == runtime.context.email_address and password == runtime.context.password:
        return Command(update={
            "authenticated": True, 
            "messages": [ToolMessage(
                "Successfully authenticated", 
                tool_call_id=runtime.tool_call_id)]
        })
    else:
        return Command(update={
            "authenticated": False,
            "messages": [ToolMessage(
                "Authentication failed", 
                tool_call_id=runtime.tool_call_id)]
        })

In [5]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Allow read inbox and send email tools only if user provides correct email and password"""

    authenticated = request.state.get("authenticated")
    
    if authenticated:
        tools = [check_inbox, send_email]
    else:
        tools = [authenticate]

    request = request.override(tools=tools) 
    return handler(request)

In [6]:
from langchain.agents.middleware import dynamic_prompt

authenticated_prompt = "You are a helpful assistant that can check the inbox and send emails."
unauthenticated_prompt = "You are a helpful assistant that can authenticate users."

@dynamic_prompt
def dynamic_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on authentication status"""
    authenticated = request.state.get("authenticated")

    if authenticated:
        return authenticated_prompt
    else:
        return unauthenticated_prompt

In [7]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

agent = create_agent(
    "gpt-5-nano",
    tools=[authenticate, check_inbox, send_email],
    checkpointer=InMemorySaver(),
    state_schema=AuthenticatedState,
    context_schema=EmailContext,
    middleware=[
        dynamic_tool_call, 
        dynamic_prompt,
        HumanInTheLoopMiddleware(
            interrupt_on={
                "authenticate": False,
                "check_inbox": False,
                "send_email": True,
            })
        ]
    )

In [8]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="eliel@example.com, password123")]},
    context=EmailContext(),
    config=config
)

print(response['messages'][-1].content)

/home/eliel/projetos-estudos/langchain-course/exercices-langchain/langchain-examples/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=EmailContext(email_addres... password='password123'), input_type=EmailContext])
  return self.__pydantic_serializer__.to_python(


I found a new message in your inbox:

From: Jane (jane@example.com)
To: Julie
Subject: (not shown in the snippet)
Message: "Hi Julie, I'm going to be in town next week and was wondering if we could grab a coffee? - best, Jane (jane@example.com)"

Would you like me to respond? If so, who should the reply come from (Julie or Eliel), and to whom should it be sent? If you want, I can draft a reply you can send. Here are a couple of options you could use:

Option A: Reply as Julie to Jane (friendly, available next week)
Subject: Re: coffee next week
Hi Jane,
Great to hear from you! I’d love to grab coffee. Are you free Tuesday or Wednesday morning next week? I can suggest a couple of spots near downtown, or you can tell me your preference.
Looking forward to it,
Julie

Option B: Neutral reply requesting details
Subject: Re: coffee next week
Hi Jane,
Thanks for reaching out. I’m in town next week—what days/times work for you? If you want, I can suggest a few coffee spots.
Best,
Julie

If you

In [10]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

KeyError: '__interrupt__'

In [11]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}  # or "reject"
    ), 
    config=config # Same thread ID to resume the paused conversation
)

print(response["messages"][-1].content)

I found a new message in your inbox:

From: Jane (jane@example.com)
To: Julie
Subject: (not shown in the snippet)
Message: "Hi Julie, I'm going to be in town next week and was wondering if we could grab a coffee? - best, Jane (jane@example.com)"

Would you like me to respond? If so, who should the reply come from (Julie or Eliel), and to whom should it be sent? If you want, I can draft a reply you can send. Here are a couple of options you could use:

Option A: Reply as Julie to Jane (friendly, available next week)
Subject: Re: coffee next week
Hi Jane,
Great to hear from you! I’d love to grab coffee. Are you free Tuesday or Wednesday morning next week? I can suggest a couple of spots near downtown, or you can tell me your preference.
Looking forward to it,
Julie

Option B: Neutral reply requesting details
Subject: Re: coffee next week
Hi Jane,
Thanks for reaching out. I’m in town next week—what days/times work for you? If you want, I can suggest a few coffee spots.
Best,
Julie

If you

In [12]:
from pprint import pprint

pprint(response)

{'authenticated': True,
 'messages': [HumanMessage(content='eliel@example.com, password123', additional_kwargs={}, response_metadata={}, id='284f805e-d782-4839-9d51-3d7671466e73'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 158, 'prompt_tokens': 151, 'total_tokens': 309, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cxf7rgdQuunTYbTzKU5smPqpzf7nC', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bb902-0373-7121-b943-f87c314a1303-0', tool_calls=[{'name': 'authenticate', 'args': {'email': 'eliel@example.com', 'password': 'password123'}, 'id': 'call_pIa1cohMGLVXvhAtfzN1F2Kb', 'type': 'tool_call